In [ ]:
#|default_exp lsd
#|export

from perturbative_llm_cognition import core

import torch
import torch.nn.functional as F

In [ ]:
#|export

class LSDPerturbedLLM:

    def __init__(
        self, 
        layer_start: int = 21,                              # Start layer to perturb    
        layer_end: int = 30,                                # Final layer to perturb
        attention_temperature_target: float = 1.20,         # Temperature > 1 flattens attention (scores /= temperature)
        attention_diagonal_penalty_target: float = 0.40,    # Subtract at most-recent key at decode (q_len==1)
        swiglu_skew_target: float = 0.15,                   # Magnitude skew exponent
        swiglu_noise_target: float = 0.12,                  # Structured noise scale (zero-mean) <- better name required
        js_tolerance: float = 0.12,                         # Tolerated JS divergence before clamping gets strong
        js_softness: float = 0.02,                          # Sigmoid softness around the tolerance <- better name required
        divergence_smoothing: float = 0.8,                              # EMA smoothing for JS
        teacher_blend_min: float = 0.05,                    # Min teacher blend weight
        teacher_blend_max: float = 0.60,                    # Max teacher blend weight when drift is high
        ):

        self.layer_start = min(layer_start, 0)
        self.layer_end = max(layer_end, 31)
        self.attention_temperature_target = max(1.0, attention_temperature_target)
        self.attention_diagonal_penalty_target = max(0.0, attention_diagonal_penalty_target)
        self.swiglu_skew_target = max(0.0, swiglu_skew_target)
        self.swiglu_noise_target = max(0.0, swiglu_noise_target)
        self.js_tolerance = max(0.0, js_tolerance)
        self.js_softness = max(0.0, js_softness)
        self.divergence_smoothing = max(0.0, divergence_smoothing)
        self.teacher_blend_min = max(0.0, teacher_blend_min)
        self.teacher_blend_max = min(1.0, teacher_blend_max)

        if self.teacher_blend_min > self.teacher_blend_max:
            raise ValueError('Invalid teacher parameters')

        self.set_perturbation_parameters(
            attention_temperature=attention_temperature_target,
            attention_diagonal_penalty=attention_diagonal_penalty_target,
            teacher_blend=teacher_blend_min,
            #swiglu_skew_target=swiglu_skew_target,
            #swiglu_noise_target=swiglu_noise_target,
        )

        self.tokenizer, self.model = core.load_tokenizer_and_model()
        self.model.config.attn_implementation = 'eager'
        self.device = self.model.device

    def set_perturbation_parameters(self, **kwargs):
        """
        Clamps all perturbation parameters to valid ranges.
        """

        for parameter, value in kwargs.items():
            if parameter in ['attention_temperature']:
                setattr(self, parameter, max(1.0, value))
            elif parameter in ['teacher_blend']:
                setattr(self, parameter, max(0.0, min(1.0, value)))
            else:
                setattr(self, parameter, max(0.0, value))

    def zero_perturbation_parameters(self):
        #reset all key perturbation parameters
        self.attention_temperature = 1.0
        self.attention_diagonal_penalty = 0.0
        #self.swiglu_skew = 0.0
        #self.swiglu_noise = 0.0

    def is_target_layer(self, index: int) -> bool:
        #may be redundant
        return self.layer_start <= index <= self.layer_end

    def apply_perturbation(self):
        self.apply_attention_perturbation()


    def apply_attention_perturbation(self):        
        for i, block in enumerate(self.model.model.layers):
            block.self_attn.layer_idx = i

            if not self.is_target_layer(i):
                continue

            attention = block.self_attn
            original_forward = attention.forward

            attention.forward = self.__get_wrapped_attention_forward(original_forward)# bind per-layer to avoid late-binding bugs

    def __get_wrapped_attention_forward(self, original_forward):
        def wrapped_forward(*args, **kwargs):
            # patch F.softmax only for this call
            original_softmax = F.softmax

            def modified_softmax(x, dimensions=None, *_args, **_kwargs):
                # Expect logits shape [batch, heads, q_len, k_len] and softmax along last dim
                if x.dim() == 4 and (dimensions == -1 or dimensions == 3):
                    y = x

                    # temperature: divide logits (skip if == 1.0)
                    tau = getattr(self, "attention_temperature", 1.0)
                    if tau != 1.0:
                        y = y / tau

                    # decode step: q_len == 1 -> penalize last key
                    diag = getattr(self, "attention_diagonal_penalty", 0.0)
                    if diag != 0.0 and y.size(-2) == 1:
                        y = y.clone()
                        y[..., -1] = y[..., -1] - diag

                    return original_softmax(y, dim=dimensions, *_args, **_kwargs)

                # otherwise, leave it alone
                return original_softmax(x, dim=dimensions, *_args, **_kwargs)

            F.softmax = modified_softmax
            try:
                return original_forward(*args, **kwargs)
            finally:
                F.softmax = original_softmax

        return wrapped_forward

    @staticmethod
    def js_divergence(p, q, eps=1e-8):
        m = 0.5 * (p + q)
        def kl(a,b):
            return (a * (torch.log(a+eps) - torch.log(b+eps))).sum(-1)
        return 0.5*kl(p, m) + 0.5*kl(q, m)

    def step(self, pert_logits, base_logits):
        # Distributions
        p_base = torch.softmax(base_logits, dim=-1)
        p_pert = torch.softmax(pert_logits, dim=-1)
        # JS divergence (batch-mean scalar)
        d = self.js_divergence(p_pert, p_base).mean().item()
        # EMA
        beta = self.divergence_smoothing
        self.running_divergence = beta * self.running_divergence + (1 - beta) * d
        # Gain (higher drift → smaller g)
        gain = torch.sigmoid(torch.tensor((self.js_tolerance - self.running_divergence) / self.js_softness)).item()

        # Effective strengths
        attention_temperature  = 1.0 + (self.attention_temperature_target - 1.0) * gain
        attention_diagonal_penalty = self.attention_diagonal_penalty_target * gain
        #skew_eff = self.cfg.swiglu_skew_target * g
        #noise_eff= self.cfg.swiglu_noise_target * g

        # Teacher blend increases as drift grows (g small)
        teacher_blend = self.teacher_blend_min + (1 - gain) * (self.teacher_blend_max - self.teacher_blend_min)
        return {
            "attention_temperature": float(attention_temperature),
            "attention_diagonal_penalty": float(attention_diagonal_penalty),
            #"skew_eff": float(skew_eff),
            #"noise_eff": float(noise_eff),
            "teaching_blend": float(teacher_blend),
            #"ema_js": float(self.running_divergence),
            #"gain": float(gain),
        }


    @torch.no_grad()
    def generate_with_leash(self,
                            prompt: str,
                            max_new_tokens: int = 128,
                            temperature: float = 0.7,
                            top_p: float = 0.95):


        input_ids = self.tokenizer(prompt, return_tensors="pt").to(self.device)['input_ids']
        generated = input_ids
        past_key_values = None

        self.running_divergence = 0.0

        # We run two forwards per step: base (no perturb), then perturbed (with current state)
        for step in range(max_new_tokens):
            # ---- BASE pass (perturbation off)
            
            perturbed_output = self.model(
                input_ids=generated, 
                use_cache=True, 
                past_key_values=past_key_values, 
                output_hidden_states=True)

            perturbed_logits = perturbed_output.logits[:, -1, :]

            self.zero_perturbation_parameters()
            base_output = self.model(
                input_ids=generated, 
                use_cache=True, 
                past_key_values=past_key_values, 
                output_hidden_states=True)

            
            base_logits = base_output.logits[:, -1, :]


            next_step = self.step(perturbed_logits, base_logits)
            self.set_perturbation_parameters(**next_step)
            past_key_values = perturbed_output.past_key_values


            # ---- Final logits for sampling: teacher blend
            final_logits = (1 - self.teacher_blend) * perturbed_logits + self.teacher_blend * base_logits

            # ---- Sample
            final_logits = final_logits / max(1e-5, temperature)
            probs = torch.softmax(final_logits, dim=-1)
            if top_p < 1.0:
                sorted_probs, sorted_idx = torch.sort(probs, descending=True)
                cum = torch.cumsum(sorted_probs, dim=-1)
                mask = cum > top_p
                mask[..., 1:] = mask[..., :-1].clone()
                mask[..., 0] = False
                sorted_probs[mask] = 0
                sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True).clamp_min(1e-9)
                next_idx = torch.multinomial(sorted_probs, num_samples=1)
                next_token = torch.gather(sorted_idx, -1, next_idx)
            else:
                next_token = torch.multinomial(probs, num_samples=1)

            generated = torch.cat([generated, next_token], dim=-1)

        return self.tokenizer.decode(generated[0], skip_special_tokens=True)

    #dtype = torch.bfloat16
    #force_eager_attention: bool = True force either way


In [ ]:
#|test
model = LSDPerturbedLLM(
    layer_start=21,                              # Start layer to perturb
    layer_end=30,                               # Final layer to perturb  
    attention_temperature_target=1.20,          # Temperature > 1 flattens attention
    attention_diagonal_penalty_target=0.40,     # Penalty for most recent key
    js_tolerance=0.12,                          # Tolerated JS divergence
    teacher_blend_min=0.05,                     # Min teacher blend weight
    teacher_blend_max=0.60,                     # Max teacher blend weight
)

# Generate text with the leash mechanism
prompt = 'Describe what you see looking out at the sea'
print(f'Prompt: {prompt}')
response = model.generate_with_leash(
    prompt=prompt,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.95
)

print(f'Response: {response}')
